In [1]:
## Useful libraries
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import os
import copy
import pickle
from urllib.request import urlretrieve
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from sklearn.preprocessing import MinMaxScaler
from matplotlib.colors import TwoSlopeNorm

from cycler import cycler
import seaborn as sns

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# for mac GPU 
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

# defining folder for conveniance
folder = "" #"C:\Users\luc\Downloads\GitHub\FLOODS3_DSAIE\raw_datasets\DEM"

device

device(type='mps')

In [2]:
def normalize_dataset(dataset, scaler_x, scaler_y):
    min_x, max_x = scaler_x.data_min_[0], scaler_x.data_max_[0]
    min_y, max_y = scaler_y.data_min_[0], scaler_y.data_max_[0]
    normalized_dataset = []
    for idx in range(len(dataset)):
        x = dataset[idx][0]
        y = dataset[idx][1]
        norm_x = (x - min_x) / (max_x - min_x)
        norm_y = (y - min_y) / (max_y - min_y)
        normalized_dataset.append((norm_x, norm_y))
    return normalized_dataset
# Normalize the inputs and outputs using training dataset
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

**Data Set**

Load data

Put in the right format 

Split training / validation data

In [3]:
length_train_val_data = 80

DEM_train_val = torch.zeros((length_train_val_data, 64, 64))

training = 0.8
validation = 0.2

for k in range(length_train_val_data):
    DEM = np.genfromtxt(f"{folder}raw_datasets/DEM/DEM_{k+1}.txt")
    DEM_t = torch.as_tensor(DEM, dtype=torch.float32)
    
    
    for x, y, elevation in zip(DEM_t[:,0], DEM_t[:,1], DEM_t[:,2]):
        #Convert coordinates to indices in the 64x64 tensor
        i = int((y - 50) / 100)
        j = int((x - 50) / 100)
        # Assign the elevation value to the corresponding position in the tensor
        DEM_train_val[k, i, j] = elevation

In [4]:
VX_train_val = torch.zeros((length_train_val_data, 97, 64, 64))

training = 0.8
validation = 0.2

for k in range(length_train_val_data):
    VX = np.genfromtxt(f"{folder}raw_datasets/VX/VX_{k+1}.txt")
    VX_t = torch.as_tensor(VX, dtype=torch.float32)
    
    for x, y, elevation in zip(VX_t[:,0], VX_t[:,1], VX_t[:,2]):
        # Convert coordinates to indices in the 64x64 tensor
        i = int((y - 50) / 100)
        j = int((x - 50) / 100)

    # Assign the elevation value to the corresponding position in the tensor
    VX_train_val[k, :, i, j] = elevation

In [5]:
VY_train_val = torch.zeros((length_train_val_data, 97, 64, 64))



for k in range(length_train_val_data):
    VY = np.genfromtxt(f"{folder}raw_datasets/VY/VY_{k+1}.txt")
    VY_t = torch.as_tensor(VY, dtype=torch.float32)
    
    for x, y, elevation in zip(VY_t[:,0], VY_t[:,1], VY_t[:,2]):
        # Convert coordinates to indices in the 64x64 tensor
        i = int((y - 50) / 100)
        j = int((x - 50) / 100)

    # Assign the elevation value to the corresponding position in the tensor
    VY_train_val[k, i, j] = elevation

In [6]:
WD_train_val = torch.zeros((length_train_val_data, 97, 64, 64))

for i in range(length_train_val_data):
    WD = np.genfromtxt(f"{folder}raw_datasets/WD/WD_{i+1}.txt")
    WD_t = torch.as_tensor(WD, dtype=torch.float32)
    for k in range(97):
        wd = WD_t[k].reshape((64,64))
        wd = torch.as_tensor(wd)

    WD_train_val[i, k] = wd

In [7]:
# # Assuming you have a tensor 'WD_train_val' with shape (length_train_val_data, 97, 64, 64)
# WD_train_val_reshaped = torch.zeros((length_train_val_data, 24, 64, 64))

# for i in range(length_train_val_data):
#     WD = np.genfromtxt(f"C:/Users/luc/Downloads/GitHub/FLOODS3_DSAIE/raw_datasets/WD//WD_{i+1}.txt")
#     WD_t = torch.as_tensor(WD, dtype=torch.float32)

#     for j in range(24):
#         # Extract a 2-hour interval from the original 97 time points
#         start_index = j * 4  # Each 2-hour interval has 4 time points (assuming 30 minutes intervals)
#         end_index = (j + 1) * 4
#         wd = WD_t[j].reshape((64,64))
#         wd = torch.tensor(wd)
#         # Average or concatenate the data over the 2-hour interval, depending on your requirement
#         wd_interval = torch.mean(wd[start_index:end_index], dim=0)  # You can use other aggregation functions if needed

#         WD_train_val_reshaped[i, j] = wd_interval

In [8]:
DEM_train_val_ex = DEM_train_val.unsqueeze(1).expand(-1, 97, -1, -1)

input_train_dataset = torch.stack((DEM_train_val_ex, WD_train_val, VX_train_val, VY_train_val)).permute(1, 2, 0, 3, 4)
print(input_train_dataset.shape)

torch.Size([80, 97, 4, 64, 64])


**Normalize data**

In [9]:
# Split dataset into train, validation, and testing

train_percnt = 0.8
train_size = int(train_percnt * length_train_val_data)
val_size = length_train_val_data - train_size
train_dataset, val_dataset = random_split(input_train_dataset, [train_size, val_size])

In [10]:
def normalize_dataset(dataset, scaler_x, scaler_y):
    min_x, max_x = scaler_x.data_min_[0], scaler_x.data_max_[0]
    min_y, max_y = scaler_y.data_min_[0], scaler_y.data_max_[0]
    normalized_dataset = []
    for idx in range(len(dataset)):
        x = dataset[idx][0]
        y = dataset[idx][1]
        norm_x = (x - min_x) / (max_x - min_x)
        norm_y = (y - min_y) / (max_y - min_y)
        normalized_dataset.append((norm_x, norm_y))
    return normalized_dataset
# Normalize the inputs and outputs using training dataset
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

In [11]:
for idx in range(len(train_dataset)):
    scaler_x.partial_fit(train_dataset[idx][0].reshape(input_train_dataset.size(2), -1).T.cpu())
    scaler_y.partial_fit(train_dataset[idx][1].reshape(-1, 1).cpu())


In [12]:
length_test_data = 20

DEM_test = torch.zeros((length_test_data, 64, 64))

for k in range(length_test_data):
    DEM = np.genfromtxt(f"{folder}raw_datasets/VY/VY_{k+1}.txt")
    #"C:\Users\luc\Downloads\GitHub\FLOODS3_DSAIE\raw_datasets\DEM"
    DEM_t = torch.as_tensor(DEM, dtype=torch.float32)
    
    
    for x, y, elevation in zip(DEM_t[:,0], DEM_t[:,1], DEM_t[:,2]):
        #Convert coordinates to indices in the 64x64 tensor
        i = int((y - 50) / 100)
        j = int((x - 50) / 100)
        # Assign the elevation value to the corresponding position in the tensor
        DEM_test[k, i, j] = elevation

In [13]:
VX_test = torch.zeros((length_test_data, 97, 64, 64))

for k in range(length_test_data):
    VX = np.genfromtxt(f"{folder}raw_datasets/VY/VY_{k+1}.txt")
    #"C:\Users\luc\Downloads\GitHub\FLOODS3_DSAIE\raw_datasets\DEM"
    VX_t = torch.as_tensor(DEM, dtype=torch.float32)
    
    
    for x, y, elevation in zip(VX_t[:,0], VX_t[:,1], VX_t[:,2]):
        #Convert coordinates to indices in the 64x64 tensor
        i = int((y - 50) / 100)
        j = int((x - 50) / 100)
        # Assign the elevation value to the corresponding position in the tensor
        VX_test[k, i, j] = elevation

In [14]:
VY_test = torch.zeros((length_test_data, 97, 64, 64))

for k in range(length_test_data):
    VY = np.genfromtxt(f"raw_datasets/VY/VY_{k+1}.txt")
    #"C:\Users\luc\Downloads\GitHub\FLOODS3_DSAIE\raw_datasets\DEM"
    VY_t = torch.as_tensor(VY, dtype=torch.float32)
    
    
    for x, y, elevation in zip(VY_t[:,0], VY_t[:,1], VY_t[:,2]):
        #Convert coordinates to indices in the 64x64 tensor
        i = int((y - 50) / 100)
        j = int((x - 50) / 100)
        # Assign the elevation value to the corresponding position in the tensor
        VY_test[k, i, j] = elevation

In [15]:
WD_test = torch.zeros((length_test_data, 97, 64, 64))

for k in range(length_test_data):
    WD = np.genfromtxt(f"{folder}raw_datasets/VY/VY_{k+1}.txt")
    #"C:\Users\luc\Downloads\GitHub\FLOODS3_DSAIE\raw_datasets\DEM"
    WD_t = torch.as_tensor(WD, dtype=torch.float32)
    
    
    for x, y, elevation in zip(WD_t[:,0], WD_t[:,1], WD_t[:,2]):
        #Convert coordinates to indices in the 64x64 tensor
        i = int((y - 50) / 100)
        j = int((x - 50) / 100)
        # Assign the elevation value to the corresponding position in the tensor
        WD_test[k, i, j] = elevation

In [16]:
DEM_test_ex = DEM_test.unsqueeze(1).expand(-1, 97, -1, -1)

test_dataset = torch.stack((DEM_test_ex, WD_test, VX_test, VY_test)).permute(1, 2, 0, 3, 4)
print(test_dataset.shape)

torch.Size([20, 97, 4, 64, 64])


In [17]:
normalized_train_dataset = normalize_dataset(input_train_dataset, scaler_x, scaler_y)
normalized_test_dataset = normalize_dataset(test_dataset, scaler_x, scaler_y)

**Simple CNN Model**

In [18]:
class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1, bias=False, batch_norm=True):
        super().__init__()
        
        layers = [nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding, bias=bias)]
        if batch_norm:
            layers.append(nn.BatchNorm2d(num_features=out_channels))
        layers.append(nn.PReLU())
        layers.append(nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=padding, bias=bias))
                
        self.cnnblock = nn.Sequential(*layers)

    def forward(self, x):
        return self.cnnblock(x)

class Encoder(nn.Module):
    def __init__(self, channels=[32, 64, 128], kernel_size=3, padding=1, bias=False, batch_norm=True):
        super().__init__()

        self.enc_blocks = nn.ModuleList([
            CNNBlock(channels[block], channels[block+1], kernel_size, padding, bias, 
                     batch_norm=batch_norm) 
            for block in range(len(channels)-1)]
            )
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
    def forward(self, x):
        outs = []
        for block in self.enc_blocks:
            x = block(x)
            outs.append(x)
            x = self.pool(x)
        return outs

class Decoder(nn.Module):
    def __init__(self, channels=[128, 64, 32], kernel_size=3, padding=1, bias=False, batch_norm=True):
        super().__init__()
        self.channels = channels
        self.upconvs = nn.ModuleList([
            nn.ConvTranspose2d(channels[block], channels[block+1], kernel_size=2, padding=0, stride=2) 
            for block in range(len(channels)-1)]
            )
        self.dec_blocks = nn.ModuleList([
            CNNBlock(channels[block], channels[block+1], kernel_size, padding, bias, 
                     batch_norm=batch_norm)
             for block in range(len(channels)-1)]
             )
        
    def forward(self, x, x_skips):
        for i in range(len(x_skips)):
            x = self.upconvs[i](x)
            x = torch.cat((x, x_skips[-(1+i)]), dim=1)
            x = self.dec_blocks[i](x)

        x = self.dec_blocks[-1](x)
        return x
    
class CNN(nn.Module):
    def __init__(self, node_features, out_dim=3, n_downsamples=3, initial_hid_dim=64, batch_norm=True, 
                 bias=True):
        super(CNN, self).__init__()
        hidden_channels = [initial_hid_dim*2**i for i in range(n_downsamples)]
        encoder_channels = [node_features]+hidden_channels
        decoder_channels = list(reversed(hidden_channels))+[out_dim]

        self.encoder = Encoder(encoder_channels, kernel_size=3, padding=1, 
                               bias=bias, batch_norm=batch_norm)
        self.decoder = Decoder(decoder_channels, kernel_size=3, padding=1, 
                               bias=bias, batch_norm=batch_norm)
        
    def forward(self, x):
        x = x.unsqueeze(0)
        x = self.encoder(x)
        x = self.decoder(x[-1], x[:-1])
        x = nn.Sigmoid()(x)
        return x



In [19]:
node_features = 4 #train_dataset[0][0].shape[0]
print(node_features)
model = CNN(node_features=node_features, n_downsamples=4, initial_hid_dim=32, 
            batch_norm=True, bias=True)

4


**Training**

How to make sure there are timesteps?

In [20]:
def train_epoch(model, loader, optimizer, device='cpu'):
    model.to(device)
    model.train() # specifies that the model is in training mode

    losses = []

    # I do not know if this is the way to go, but now it calculates and backpropagates for each timestep in each batch
    for batch in loader:
        for t in range(1, len(batch[0])):
            x = batch[0][t-1]
            y = batch[0][t].unsqueeze(1)
        
            # Model prediction
            preds = model(x)
            # MSE loss function
            loss = nn.MSELoss()(preds, y)
            losses.append(loss.cpu().detach())
            # Backpropagate and update weights
            loss.backward()   # compute the gradients using backpropagation
            optimizer.step()  # update the weights with the optimizer
            optimizer.zero_grad(set_to_none=True)   # reset the computed gradients

    losses = np.array(losses).mean()
    print('train epoch complete')

    return losses

In [21]:
def evaluation(model, loader, device='cpu'):
    model.to(device)
    model.eval() # specifies that the model is in evaluation mode
    batch_n = 0
    losses = []
    
    for batch in loader:
        for t in range(1, len(batch[0])):
            x = batch[0][t-1]
            y = batch[0][t].unsqueeze(1)

            # Model prediction
            preds = model(x)

            # MSE loss function
            loss = nn.MSELoss()(preds, y)
            losses.append(loss.cpu().detach())
        loss = np.array(losses).mean()

    return loss

In [22]:
# Set training parameters
learning_rate = 0.001
batch_size = 1
num_epochs = 10

# Create the optimizer to train the neural network via back-propagation
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)

# Create the training and validation dataloaders to "feed" data to the model in batches



train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
# test_loader = DataLoader(normalized_test_dataset, batch_size=batch_size, shuffle=False)

In [23]:
#create vectors for the training and validation loss
train_losses = []
val_losses = []


for epoch in range(1, num_epochs+1):
    # Model training
    train_loss = train_epoch(model, train_loader, optimizer, device=device)

    # Model validation
    val_loss = evaluation(model, val_loader, device=device)

    if epoch == 1:
        best_loss = val_loss

    if val_loss<=best_loss:
        best_model = copy.deepcopy(model)
        best_loss = val_loss
        best_epoch = epoch

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    if epoch%10 == 0:
        print("epoch:",epoch, "\t training loss:", np.round(train_loss,4),
                            "\t validation loss:", np.round(val_loss,4))

model = copy.deepcopy(best_model)

RuntimeError: slow_conv2d_forward_mps: input(device='cpu') and weight(device=mps:0')  must be on the same device

In [ ]:
print("epoch:",epoch, "\t training loss:", np.round(train_loss,4),
                            "\t validation loss:", np.round(val_loss,4))
model = copy.deepcopy(best_model)

In [ ]:
plt.plot(train_losses, label='Training')
plt.plot(val_losses, label='Validation')
plt.yscale('log')
plt.title('Losses')
plt.xlabel('Epochs')
plt.legend()
plt.show()

**Testing**

I have not done this, the code below is copied does not work yet!

Model is untested!

In [ ]:
test_loss = evaluation(model, test_loader, device=device)
print(test_loss)

In [ ]:
# select one sample
data_id = 10

x = normalized_test_dataset[data_id][0].unsqueeze(0)
FAT = normalized_test_dataset[data_id][1]

# predict the FAT
pred_FAT = model(x).detach() # I

print(pred_FAT.shape)

In [ ]:
DEM = scaler_x.inverse_transform(x[0].reshape(2,-1).T.cpu())[:,0].reshape(64,64)
real_FAT = scaler_y.inverse_transform(FAT[-1].reshape(-1, 1).cpu()).reshape(64,64)
# real_FAT_2 = scaler_y.inverse_transform(FAT.reshape(96,-1).T.cpu()).reshape(64,64)

pred_FAT = scaler_y.inverse_transform(pred_FAT[0][-1].reshape(-1,1).cpu()).reshape(64,64)
# pred_FAT_2 = scaler_y.inverse_transform(pred_FAT.reshape(96,-1).T.cpu()).reshape(64,64)

print(real_FAT.shape)

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(17,5))

diff_FAT = real_FAT - pred_FAT
max_FAT = max(pred_FAT.max(), real_FAT.max())
max_diff = max(diff_FAT.max(), -diff_FAT.min())

axs[0].imshow(DEM.squeeze(), cmap='terrain', origin='lower')
axs[1].imshow(real_FAT.squeeze(), vmin = 0, vmax=max_FAT, cmap='Blues_r', origin='lower')
axs[2].imshow(pred_FAT.squeeze(), vmin = 0, vmax=max_FAT,cmap='Blues_r', origin='lower')
axs[3].imshow(diff_FAT.squeeze(), vmin=-max_diff, vmax=max_diff, cmap='RdBu', origin='lower')

plt.colorbar(plt.cm.ScalarMappable(norm=plt.Normalize(vmin = DEM.min(), vmax=DEM.max()),
                            cmap='terrain'), fraction=0.05, shrink=0.9, ax=axs[0])
plt.colorbar(plt.cm.ScalarMappable(norm=plt.Normalize(vmin = 0, vmax=max_FAT),
                            cmap='Blues_r'), fraction=0.05, shrink=0.9, ax=axs[1])
plt.colorbar(plt.cm.ScalarMappable(norm=plt.Normalize(vmin = 0, vmax=max_FAT),
                            cmap='Blues_r'), fraction=0.05, shrink=0.9, ax=axs[2])
plt.colorbar(plt.cm.ScalarMappable(norm=TwoSlopeNorm(vmin=-max_diff, vmax=max_diff, vcenter=0),
                            cmap='RdBu'), fraction=0.05, shrink=0.9, ax=axs[3])
for ax in axs:
    ax.axis('off')

axs[0].set_title('DEM')
axs[1].set_title('Real FAT (h)')
axs[2].set_title('Predicted FAT (h)')
axs[3].set_title('Difference (h)')

plt.show()